In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
from streamlit_option_menu import option_menu
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
# Set the Streamlit page configuration
st.set_page_config(
    page_title="Global EV Charging Insights",
    page_icon="🔌",
    layout="wide",
)

In [ ]:
# Function to load and preprocess data
@st.cache_data
def load_data():
    df = pd.read_csv('Global_EV_Charging_Behavior_2024.csv')
    # Preprocess date/time columns
    df['Charging Start Time'] = pd.to_datetime(df['Charging Start Time'])
    df['Charging End Time'] = pd.to_datetime(df['Charging End Time'])
    # Extract day of week and hour of day
    df['Day of Week'] = df['Charging Start Time'].dt.day_name()
    df['Hour of Day'] = df['Charging Start Time'].dt.hour
    return df

In [ ]:
# Load the data
df = load_data()

In [ ]:
# --- SIDEBAR NAVIGATION MENU ---
with st.sidebar:
    selected = option_menu(
        menu_title="Main Menu",
        options=["Dashboard Overview", "Performance Analysis", "Predictive Model", "Geographical Analysis"],
        icons=["house", "bar-chart", "lightning", "globe"],
        menu_icon="cast",
        default_index=0,
    )

In [ ]:
# --- DASHBOARD OVERVIEW PAGE ---
if selected == "Dashboard Overview":
    st.title("🔌 Global EV Charging Behavior Dashboard")
    st.markdown("A comprehensive look into charging session data from around the world.")

    # Top-level metrics
    col1, col2, col3 = st.columns(3)
    total_sessions = df.shape[0]
    avg_duration = round(df['Charging Duration (mins)'].mean(), 2)
    total_energy = round(df['Energy Delivered (kWh)'].sum(), 2)
    avg_cost = round(df['Charging Cost ($)'].mean(), 2)

    col1.metric("Total Charging Sessions", f"{total_sessions:,}")
    col2.metric("Avg. Session Duration", f"{avg_duration} mins")
    col3.metric("Total Energy Delivered", f"{total_energy:,} kWh")
    st.metric("Avg. Cost per Session", f"${avg_cost}")

    st.markdown("---")

    # Chart for Session Outcome Distribution
    st.header("Charging Session Outcomes")
    outcome_counts = df['Charging Session Outcome'].value_counts()
    fig1, ax1 = plt.subplots(figsize=(8, 8))
    ax1.pie(outcome_counts, labels=outcome_counts.index, autopct='%1.1f%%', startangle=90, colors=plt.cm.Pastel1.colors)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    st.pyplot(fig1)

    st.markdown("---")

    # Chart for Top 10 Countries by Charging Sessions
    st.header("Geographical Insights")
    country_counts = df['Country'].value_counts().head(10)
    fig2, ax2 = plt.subplots(figsize=(12, 6))
    country_counts.plot(kind='bar', ax=ax2, color='skyblue')
    ax2.set_title('Top 10 Countries by Number of Charging Sessions')
    ax2.set_xlabel('Country')
    ax2.set_ylabel('Number of Sessions')
    plt.xticks(rotation=45)
    st.pyplot(fig2)

In [ ]:
# --- PERFORMANCE ANALYSIS PAGE ---
elif selected == "Performance Analysis":
    st.title("📈 Performance and Efficiency Analysis")
    st.markdown("Explore charging performance by station type and vehicle manufacturer.")

    # Session efficiency by Station Type
    st.header("Energy Delivered by Charging Station Type")
    energy_by_type = df.groupby('Charging Station Type')['Energy Delivered (kWh)'].mean().sort_values(ascending=False)
    fig3, ax3 = plt.subplots(figsize=(10, 6))
    energy_by_type.plot(kind='bar', ax=ax3, color='salmon')
    ax3.set_title('Average Energy Delivered per Session by Station Type')
    ax3.set_xlabel('Charging Station Type')
    ax3.set_ylabel('Avg. Energy Delivered (kWh)')
    plt.xticks(rotation=0)
    st.pyplot(fig3)

    st.markdown("---")

    # Manufacturer Performance
    st.header("Manufacturer Performance")
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Avg. Duration by Manufacturer")
        duration_by_manu = df.groupby('Manufacturer')['Charging Duration (mins)'].mean().sort_values(ascending=False)
        fig4, ax4 = plt.subplots(figsize=(10, 6))
        duration_by_manu.plot(kind='barh', ax=ax4, color='lightgreen')
        ax4.set_title('Average Charging Duration by Manufacturer')
        ax4.set_xlabel('Avg. Duration (mins)')
        ax4.set_ylabel('Manufacturer')
        st.pyplot(fig4)

    with col2:
        st.subheader("Avg. Cost by Manufacturer")
        cost_by_manu = df.groupby('Manufacturer')['Charging Cost ($)'].mean().sort_values(ascending=False)
        fig5, ax5 = plt.subplots(figsize=(10, 6))
        cost_by_manu.plot(kind='barh', ax=ax5, color='orange')
        ax5.set_title('Average Charging Cost by Manufacturer')
        ax5.set_xlabel('Avg. Cost ($)')
        ax5.set_ylabel('Manufacturer')
        st.pyplot(fig5)

In [ ]:
# --- PREDICTIVE MODEL PAGE ---
elif selected == "Predictive Model":
    st.title("🔮 Simple Predictive Analysis")
    st.markdown("This model predicts **Energy Delivered** based on **Charging Duration**.")

    # Prepare data for the model
    X = df[['Charging Duration (mins)']].values.reshape(-1, 1)
    y = df['Energy Delivered (kWh)'].values

    # Create and train the model
    model = LinearRegression()
    model.fit(X, y)

    # Get model performance
    r_squared = model.score(X, y)

    st.subheader("Model Performance")
    st.info(f"The R-squared value of the model is: **{r_squared:.2f}**")
    st.write("This value indicates how well the variance in `Energy Delivered` is explained by `Charging Duration`.")

    st.markdown("---")

    # Interactive prediction
    st.header("Make a Prediction")
    duration = st.slider("Select Charging Duration (mins)", min_value=1, max_value=int(df['Charging Duration (mins)'].max()), value=60)
    predicted_energy = model.predict(np.array([[duration]]))
    st.success(f"Predicted Energy Delivered for a {duration}-minute session: **{predicted_energy[0]:.2f} kWh**")
    
    # Visualize the model
    st.header("Visualization of the Predictive Model")
    fig6, ax6 = plt.subplots(figsize=(10, 6))
    ax6.scatter(X, y, color='blue', label='Actual Data')
    ax6.plot(X, model.predict(X), color='red', linewidth=2, label='Regression Line')
    ax6.set_title('Predicted vs. Actual Energy Delivered')
    ax6.set_xlabel('Charging Duration (mins)')
    ax6.set_ylabel('Energy Delivered (kWh)')
    ax6.legend()
    st.pyplot(fig6)

In [ ]:
# --- GEOGRAPHICAL ANALYSIS PAGE ---
elif selected == "Geographical Analysis":
    st.title("🗺️ Geographical Breakdown")
    st.markdown("Explore key metrics for specific countries and cities.")

    # Add image carousel
    st.subheader("Image Gallery")
    
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        st.image("evs_at_charging_station.jpg", caption="Charging Station", use_container_width=True)
    with col2:
        st.image("people_using_ev_chargers.jpg", caption="Dashboard Analytics", use_container_width=True)
    with col3:
        st.image("ev_charging_station_types.jpg", caption="EV Charging", use_container_width=True)
    with col4:
        st.image("electric_vehicle_models.jpg", caption="Charging at Home", use_container_width=True)

    st.markdown("---")

    # Create select boxes for Country and City
    country_list = df['Country'].unique()
    selected_country = st.selectbox("Select a Country", country_list)

    # Filter data based on selected country
    filtered_df = df[df['Country'] == selected_country]

    city_list = filtered_df['City'].unique()
    selected_city = st.selectbox("Select a City", city_list)

    # Filter data based on selected city
    final_filtered_df = filtered_df[filtered_df['City'] == selected_city]

    if not final_filtered_df.empty:
        st.subheader(f"Metrics for {selected_city}, {selected_country}")
        
        # Display key metrics for the selected city and country
        city_col1, city_col2, city_col3 = st.columns(3)
        city_col1.metric("Total Sessions", f"{final_filtered_df.shape[0]:,}")
        city_col2.metric("Avg. Duration", f"{final_filtered_df['Charging Duration (mins)'].mean():.2f} mins")
        city_col3.metric("Avg. Cost", f"${final_filtered_df['Charging Cost ($)'].mean():.2f}")
        
        # Display average energy delivered
        avg_energy = final_filtered_df['Energy Delivered (kWh)'].mean()
        st.metric("Avg. Energy Delivered", f"{avg_energy:.2f} kWh")

        # Display a simple bar chart
        st.header("Charging Session Outcome Distribution")
        city_outcome_counts = final_filtered_df['Charging Session Outcome'].value_counts()
        fig7, ax7 = plt.subplots(figsize=(8, 6))
        city_outcome_counts.plot(kind='bar', ax=ax7, color='teal')
        ax7.set_title(f"Session Outcomes in {selected_city}")
        ax7.set_xlabel('Outcome')
        ax7.set_ylabel('Number of Sessions')
        plt.xticks(rotation=0)
        st.pyplot(fig7)

    else:
        st.warning("No data available for the selected country and city combination.")